In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
! pip install -q gdown
! pip install -q torchmetrics

!wget --no-check-certificate -O bully_data.zip "https://iitgoffice-my.sharepoint.com/:u:/g/personal/sjana_iitg_ac_in/EWdMrS9zHgBHnz4TTckw14kB14O8j0IbR_D-fBowyw7T7A?e=8H4Wpw&download=1"

!gdown "https://drive.google.com/uc?id=1RpvrH0OJXBnJDzAFerqBb7v1ZKLbxTG8"
!gdown "https://drive.google.com/uc?id=19oLWMCuVHye63YvKBrG7-EbW4OU_VQOQ"

!rm -rf bully_data/
!mkdir bully_data
!mkdir bully_data/data
#!mkdir bully_data/test
!unzip bully_data.zip -d bully_data/data/ | tqdm >/dev/null
!unzip Cyberbully_corrected_emotion_sentiment_v2.zip -d bully_data/ | tqdm >/dev/null

--2024-11-24 13:58:25--  https://iitgoffice-my.sharepoint.com/:u:/g/personal/sjana_iitg_ac_in/EWdMrS9zHgBHnz4TTckw14kB14O8j0IbR_D-fBowyw7T7A?e=8H4Wpw&download=1
Resolving iitgoffice-my.sharepoint.com (iitgoffice-my.sharepoint.com)... 13.107.138.10, 13.107.136.10, 2620:1ec:8f8::10, ...
Connecting to iitgoffice-my.sharepoint.com (iitgoffice-my.sharepoint.com)|13.107.138.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/sjana_iitg_ac_in/Documents/bully_data-20240404T103912Z-001.zip?ga=1 [following]
--2024-11-24 13:58:26--  https://iitgoffice-my.sharepoint.com/personal/sjana_iitg_ac_in/Documents/bully_data-20240404T103912Z-001.zip?ga=1
Reusing existing connection to iitgoffice-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 124138703 (118M) [application/x-zip-compressed]
Saving to: 'bully_data.zip'

bully_data.zip      100%[===================>] 118.39M  25.6MB/s    in 8.5s    

2024-11-24 13:58:35 (14.0 MB/s) - 'bully

# Download the Data

## Images

## Excel Data

In [2]:
import pandas as pd
import pickle
import os

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import transforms, models
from transformers import RobertaTokenizer, RobertaModel

from torch.utils.data import random_split
from sklearn.metrics import accuracy_score, f1_score

import numpy as np
from PIL import Image

from tqdm import tqdm

In [3]:
# Load the Excel file
#df = pd.read_excel('/kaggle/working/bully_data/Cyberbully_corrected_emotion_sentiment_v2.xlsx')
df = pd.read_excel("/kaggle/working/bully_data/data/bully_data/Copy of Cyberbully_corrected_emotion_sentiment.xlsx")
df = df.drop(columns=['Unnamed: 10', 'Unnamed: 11'])
# Display the first few rows to ensure it is loaded correctly
# print(df)
df.head()

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target
0,0.jpg,Shivam @shivamishraa Girls be named naina and ...,Bully,Nonbully,Bully,Negative,Disgust,Yes,Partially-Harmful,Individual
1,1.jpg,Aaloo ke paranthe is the best breakfast Omelet...,Nonbully,Nonbully,Nonbully,Neutral,Other,No,Harmless,NaN
2,2.jpg,For Boyfriend For Bestfriend DESI ADUKT TROLLS,Bully,Bully,Nonbully,Negative,Ridicule,No,Partially-Harmful,Society
3,3.jpg,You find a new YouTuber He's funny All of his ...,Nonbully,Nonbully,Nonbully,Neutral,Sadness,No,Harmless,NaN
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Partially-Harmful,Individual


In [4]:
df_cleaned = df.dropna()
df=df_cleaned
# print(df)
df.head()

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target
0,0.jpg,Shivam @shivamishraa Girls be named naina and ...,Bully,Nonbully,Bully,Negative,Disgust,Yes,Partially-Harmful,Individual
2,2.jpg,For Boyfriend For Bestfriend DESI ADUKT TROLLS,Bully,Bully,Nonbully,Negative,Ridicule,No,Partially-Harmful,Society
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Partially-Harmful,Individual
5,5.jpg,what if we use 100% of our brain?,Bully,Bully,Nonbully,Negative,Surprise,No,Partially-Harmful,Individual
6,6.jpg,If the opposite of Con is Pro Is Congress the ...,Bully,Nonbully,Bully,Negative,Sadness,No,Partially-Harmful,Organization


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3105 entries, 0 to 5864
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Img_Name        3105 non-null   object
 1   Img_Text        3105 non-null   object
 2   Img_Text_Label  3105 non-null   object
 3   Img_Label       3105 non-null   object
 4   Text_Label      3105 non-null   object
 5   Sentiment       3105 non-null   object
 6   Emotion         3105 non-null   object
 7   Sarcasm         3105 non-null   object
 8   Harmful_Score   3105 non-null   object
 9   Target          3105 non-null   object
dtypes: object(10)
memory usage: 266.8+ KB


In [6]:
df.describe()

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target
count,3105,3105,3105,3105,3105,3105,3105,3105,3105,3105
unique,3105,3036,2,2,2,3,10,2,3,4
top,6005.jpg,a head diaper is required when you have shit f...,Bully,Nonbully,Nonbully,Negative,Disgust,No,Partially-Harmful,Individual
freq,1,6,3076,2427,1640,2310,865,1577,3062,2430


In [7]:
print("Columns: ", df.columns.tolist())
print(f"{df.columns.tolist()[2]}: {df[df.columns.tolist()[2]].unique()} ")
print(f"{df.columns.tolist()[3]}: {df[df.columns.tolist()[3]].unique()} ")
print(f"{df.columns.tolist()[4]}: {df[df.columns.tolist()[4]].unique()} ")
print(f"{df.columns.tolist()[5]}: {df[df.columns.tolist()[5]].unique()} ")
print(f"{df.columns.tolist()[6]}: {df[df.columns.tolist()[6]].unique()} ")
print(f"{df.columns.tolist()[7]}: {df[df.columns.tolist()[7]].unique()} ")
print(f"{df.columns.tolist()[8]}: {df[df.columns.tolist()[8]].unique()} ")
print(f"{df.columns.tolist()[9]}: {df[df.columns.tolist()[9]].unique()} ")



Columns:  ['Img_Name', 'Img_Text', 'Img_Text_Label', 'Img_Label', 'Text_Label', 'Sentiment', 'Emotion', 'Sarcasm', 'Harmful_Score', 'Target']
Img_Text_Label: ['Bully' 'Nonbully'] 
Img_Label: ['Nonbully' 'Bully'] 
Text_Label: ['Bully' 'Nonbully'] 
Sentiment: ['Negative' 'Neutral' 'Positive'] 
Emotion: ['Disgust' 'Ridicule' 'Sadness' 'Surprise' 'Angry' 'Anticipation'
 'Happiness' 'Other' 'Trust' 'Fear'] 
Sarcasm: ['Yes' 'No'] 
Harmful_Score: ['Partially-Harmful' 'Very-Harmful' 'Harmless'] 
Target: ['Individual' 'Society' 'Organization' 'Community'] 


In [9]:
df.iloc[761]

Img_Name                                                1098.png
Img_Text          show me your bare naked ankles you dirty whore
Img_Text_Label                                             Bully
Img_Label                                               Nonbully
Text_Label                                                 Bully
Sentiment                                               Negative
Emotion                                                 Surprise
Sarcasm                                                       No
Harmful_Score                                  Partially-Harmful
Target                                                Individual
Name: 1098, dtype: object

In [8]:
# Assuming your DataFrame is called df and contains the 'img_id' column
# Assuming image paths are in a directory (img_dir) and filenames correspond to 'img_id'

img_dir = "/kaggle/working/bully_data/data/bully_data/"

# Define a function to check if the image size is zero
def is_zero_size(img_id, img_dir):
    img_path = os.path.join(img_dir, img_id)
    return os.path.exists(img_path) and os.path.getsize(img_path) == 0

# Filter out rows with zero-size images
df['is_zero_size'] = df['Img_Name'].apply(lambda img_id: is_zero_size(img_id, img_dir))
df_filtered = df[df['is_zero_size'] == False].drop(columns='is_zero_size')

# Now, df_filtered contains only rows with non-zero-size images
# print(df_filtered)
df=df_filtered
df_cleaned = df[df['Img_Name'] != '2644.jpg']
df=df_cleaned

df.head()

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target
0,0.jpg,Shivam @shivamishraa Girls be named naina and ...,Bully,Nonbully,Bully,Negative,Disgust,Yes,Partially-Harmful,Individual
2,2.jpg,For Boyfriend For Bestfriend DESI ADUKT TROLLS,Bully,Bully,Nonbully,Negative,Ridicule,No,Partially-Harmful,Society
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Partially-Harmful,Individual
5,5.jpg,what if we use 100% of our brain?,Bully,Bully,Nonbully,Negative,Surprise,No,Partially-Harmful,Individual
6,6.jpg,If the opposite of Con is Pro Is Congress the ...,Bully,Nonbully,Bully,Negative,Sadness,No,Partially-Harmful,Organization


In [9]:
df.shape

(3078, 10)

In [12]:
# import os
# import cv2
# bad_list=[]
# data_dir='/kaggle/working/bully_data/data/bully_data'
# subdir_list=os.listdir(data_dir) # create a list of the sub directories in the directory ie train or test

# for img_file in subdir_list:
#     img_file_path = os.path.join(data_dir, img_file)
#     index = img_file_path.rfind('.')
#     file_ext = img_file_path[index+1:]
    
#     if file_ext not in ['jpg', 'png', 'bmp', 'gif', 'jpeg']:
#         print(f'file {img_file_path}  has an invalid extension {file_ext}')
#         bad_list.append(img_file)
#     else:
#         try:
#             #print(img_file_path)
#             #img = Image.open(img_file_path).convert('RGB')
#             image = Image.open("/kaggle/working/bully_data/data/bully_data/81.jpg").convert('RGB')
#         except Exception as e:
#             bad_list.append(img_file)


In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Make Dataset

In [13]:
class MemeDataset(Dataset):
    def __init__(self, dataframe, transform):
        self.dataframe = dataframe
        self.transform = transform
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.img_folder = '/kaggle/working/bully_data/data/bully_data'
        
        # Define label mappings
        self.text_label_mapping = {
            "Bully": 1,
            "Nonbully": 0
        }
        
        self.sentiment_mapping = {
            "Positive":1,
            "Neutral": 0,
            "Negative": 2
        }
        
        self.emotion_mapping = {
            "Disgust": 0,
            "Ridicule": 1,
            "Sadness": 2,
            "Surprise": 3,
            "Anticipation": 4,
            "Angry": 5,
            "Happiness": 6,
            "Other": 7,
            "Trust": 8,
            "Fear": 9
        }
        
        self.sarcasm_mapping = {
            "Yes": 1,
            "No": 0
        }
        
        self.harmful_score_mapping = {
            "Harmless": 0,
            "Partially-Harmful": 1,
            "Very-Harmful": 2
        }
        
        self.target_mapping = {
            "Individual": 0,
            "Society": 1,
            "Organization": 2,
            "Community": 3
        }
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        # Load image
        img_name = self.dataframe.iloc[idx]['Img_Name']
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        
        # Load and tokenize text
        text = self.dataframe.iloc[idx]['Img_Text']
        inputs = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        
        # Get labels and apply mappings
        sentiment_label = torch.tensor(self.sentiment_mapping[self.dataframe.iloc[idx]['Sentiment']], dtype=torch.long)
        emotion_label = torch.tensor(self.emotion_mapping[self.dataframe.iloc[idx]['Emotion']], dtype=torch.long)
        sarcasm_label = torch.tensor(self.sarcasm_mapping[self.dataframe.iloc[idx]['Sarcasm']], dtype=torch.float)  # Binary sarcasm
        bully_label = torch.tensor(self.text_label_mapping[self.dataframe.iloc[idx]['Img_Label']], dtype=torch.long)  # Bully detection
        harmful_score_label = torch.tensor(self.harmful_score_mapping[self.dataframe.iloc[idx]['Harmful_Score']], dtype=torch.long)
        target_label = torch.tensor(self.target_mapping[self.dataframe.iloc[idx]['Target']], dtype=torch.long)
        
        return image, inputs['input_ids'].squeeze(), inputs['attention_mask'].squeeze(), sentiment_label, emotion_label, sarcasm_label, bully_label, harmful_score_label, target_label

In [15]:
# Load the dataset (assume the dataframe and image directory are available)
dataset = MemeDataset(df, transform)

# Create a DataLoader for batching
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

# Iterate through the dataloader
for batch in dataloader:
    ##print(f"Len: {len(batch)}") 
    # Further model training steps...
    break
    
    
print("Done")
    


Done


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
image = Image.open("/kaggle/working/bully_data/data/bully_data/81.jpg").convert('RGB')

UnidentifiedImageError: cannot identify image file '/kaggle/working/bully_data/data/bully_data/81.jpg'

# Model

In [14]:
# This Model Converges Faster
class MultiTaskModel(nn.Module):
    def __init__(self):
        super(MultiTaskModel, self).__init__()
        
        # Visual branch (CNN)
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final classification layer
        
        # Textual branch (RoBERTa)
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        
        # Shared fully connected layers
        self.fc_shared = nn.Sequential(
            nn.Linear(2048 + 768, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # Task-specific heads
        self.sentiment_head = nn.Linear(512, 3)  # Sentiment: 3 classes
        self.emotion_head = nn.Linear(512, 10)    # Emotion: 6 classes, Its 10 not 6
        self.sarcasm_head = nn.Linear(512, 1)    # Sarcasm: binary classification
        self.bully_head = nn.Linear(512, 2)      # Cyberbullying: 2 classes
        self.fc_harmful_score = nn.Linear(512, 3)  # 3 classes for harmful score
        self.fc_target = nn.Linear(512, 4)  # 4 classes for target
    
    def forward(self, image, text_input_ids, text_attention_mask):
        # Visual features
        img_features = self.resnet(image)
        
        # Textual features
        text_outputs = self.roberta(input_ids=text_input_ids, attention_mask=text_attention_mask)
        text_features = text_outputs.pooler_output
        
        # Concatenate the visual and textual features
        combined_features = torch.cat((img_features, text_features), dim=1)
        
        # Shared layers
        shared_out = self.fc_shared(combined_features)
        
        # Task-specific outputs
        sentiment_out = self.sentiment_head(shared_out)
        emotion_out = self.emotion_head(shared_out)
        sarcasm_out = self.sarcasm_head(shared_out)
        bully_out = self.bully_head(shared_out)
        harmful_out = self.fc_harmful_score(shared_out)
        target_out = self.fc_target(shared_out)
        
        return sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_out, target_out

In [15]:
class MemeModel(nn.Module):
    def __init__(self):
        super(MemeModel, self).__init__()
        
        # Load pre-trained RoBERTa model
        self.text_model = RobertaModel.from_pretrained('roberta-base')
        
        # Load pre-trained ResNet model for image processing
        self.image_model = models.resnet18(pretrained=True)
        self.image_model.fc = nn.Linear(self.image_model.fc.in_features, 512)  # Modify the final layer
        
        # Shared fully connected layers
        self.fc_shared = nn.Sequential(
            nn.Linear(512 + 768, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        # Define fully connected layers for each task
        self.fc_sentiment = nn.Linear(256, 3)  # 3 classes for sentiment
        self.fc_emotion = nn.Linear(256, 10)  # 10 classes for emotion
        self.fc_sarcasm = nn.Linear(256, 1)  # Binary classification for sarcasm
        self.fc_bully = nn.Lineaavg_val_loss = val_loss_sum / len(val_dataloader)r(256, 2)  # Binary classification for bully detection
        self.fc_harmful_score = nn.Linear(256, 3)  # 3 classes for harmful score
        self.fc_target = nn.Linear(256, 4)  # 4 classes for target
    
    def forward(self, image, input_ids, attention_mask):
        # Process text input
        text_outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_features = text_outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token representation
        
        # Process image input
        image_features = self.image_model(image)
        
        # Concatenate text and image features
        combined_features = torch.cat((text_features, image_features), dim=1)
        
        features = self.fc_shared(combined_features)
        
        # Pass through fully connected layers for each task
        sentiment_output = self.fc_sentiment(features)
        emotion_output = self.fc_emotion(features)
        sarcasm_output = self.fc_sarcasm(features)
        bully_output = self.fc_bully(features)
        harmful_score_output = self.fc_harmful_score(features)
        target_output = self.fc_target(features)
        
        # Task-specific outputs with softmax or sigmoid
#         sentiment_output = F.softmax(self.fc_sentiment(features), dim=1)  # 3-class softmax
#         emotion_output = F.softmax(self.fc_emotion(features), dim=1)      # 10-class softmax
#         sarcasm_output = torch.sigmoid(self.fc_sarcasm(features))         # Binary sigmoid
#         bully_output = F.softmax(self.fc_bully(features), dim=1)          # Binary softmax
#         harmful_score_output = F.softmax(self.fc_harmful_score(features), dim=1)  # 3-class softmax
#         target_output = F.softmax(self.fc_target(features), dim=1)        # 4-class softmax
        
        return sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output


SyntaxError: invalid syntax (984173927.py, line 26)

In [19]:
#Example usage
model = MemeModel()
image, input_ids, attention_mask, sentiment_label, emotion_label, sarcasm_label, bully_label, harmful_score_label, target_label = dataset[0]
sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output = model(image.unsqueeze(0), input_ids.unsqueeze(0), attention_mask.unsqueeze(0))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")



Device: cuda


In [15]:
import torch.nn.functional as F

# Define the loss functions
loss_fn_sentiment = nn.CrossEntropyLoss()
loss_fn_emotion = nn.CrossEntropyLoss()
loss_fn_sarcasm = nn.BCEWithLogitsLoss()
loss_fn_bully = nn.CrossEntropyLoss()
loss_fn_harmful_score = nn.CrossEntropyLoss()
loss_fn_target = nn.CrossEntropyLoss()

In [ ]:
# # Initialize the model
# model = MemeModel()

# # Define the optimizer
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [25]:
# Load the dataset (assume the dataframe and image directory are available)
dataset = MemeDataset(df, transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split Dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize the model and move it to the GPU

# img_clip_inp_dim = 512
# vgg_inp_dim = 25088
# text_clip_inp_dim = 512
# text_roberta_inp_dim = 768

# Initialize the model
#model = MemeModel()
model = MultiTaskModel()

# Parallelize
# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model)  # Use DataParallel if multiple GPUs are available

model.to(device) 

# Initialize the Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 171MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Example usage
#model = MemeModel()
image, input_ids, attention_mask, sentiment_label, emotion_label, sarcasm_label, bully_label, harmful_score_label, target_label = dataset[0]
sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output = model(image, input_ids, attention_mask)

In [ ]:
! pip install -q torchmetrics


In [59]:
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, BinaryAccuracy, BinaryF1Score

# Initialize metrics
accuracy_sentiment = MulticlassAccuracy(num_classes=3).to(device)
f1_sentiment = MulticlassF1Score(num_classes=3, average="weighted").to(device)

accuracy_emotion = MulticlassAccuracy(num_classes=10).to(device)
f1_emotion = MulticlassF1Score(num_classes=10, average="weighted").to(device)

accuracy_sarcasm = BinaryAccuracy().to(device)
f1_sarcasm = BinaryF1Score().to(device)

# accuracy_bully = BinaryAccuracy().to(device)
# f1_bully = BinaryF1Score().to(device)
accuracy_bully = MulticlassAccuracy(num_classes=2).to(device)
f1_bully = MulticlassF1Score(num_classes=2, average="weighted").to(device)

accuracy_harmful_score = MulticlassAccuracy(num_classes=3).to(device)
f1_harmful_score = MulticlassF1Score(num_classes=3, average="weighted").to(device)

accuracy_target = MulticlassAccuracy(num_classes=4).to(device)
f1_target = MulticlassF1Score(num_classes=4, average="weighted").to(device)

In [57]:
def train_model(model, train_dataloader, val_dataloader, optimizer, losses, vals, num_epochs=10):
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        # batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
        batch_iterator = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch [{epoch+1}/{num_epochs}]')
        for batch_idx, (images, input_ids, attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels) in batch_iterator:
            # Move data to the appropriate device (GPU or CPU)
            images = images.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            sentiment_labels = sentiment_labels.to(device)
            emotion_labels = emotion_labels.to(device)
            sarcasm_labels = sarcasm_labels.to(device)
            bully_labels = bully_labels.to(device)
            harmful_score_labels = harmful_score_labels.to(device)
            target_labels = target_labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output = model(images, input_ids, attention_mask)

            # Compute the losses
            loss_sentiment = loss_fn_sentiment(sentiment_output, sentiment_labels)
            loss_emotion = loss_fn_emotion(emotion_output, emotion_labels)
            loss_sarcasm = loss_fn_sarcasm(sarcasm_output.squeeze(), sarcasm_labels.float())
            loss_bully = loss_fn_bully(bully_output, bully_labels)
            loss_harmful_score = loss_fn_harmful_score(harmful_score_output, harmful_score_labels)
            loss_target = loss_fn_target(target_output, target_labels)

            # Combine the losses
            loss = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += loss.item()
            
            # Calculate metrics
            acc_sent = accuracy_sentiment(sentiment_output, sentiment_labels)
            f1_sent = f1_sentiment(sentiment_output, sentiment_labels)
            
            acc_emo = accuracy_emotion(emotion_output, emotion_labels)
            f1_emo = f1_emotion(emotion_output, emotion_labels)
            
            acc_sarc = accuracy_sarcasm(sarcasm_output.squeeze(), sarcasm_labels)
            f1_sarc = f1_sarcasm(sarcasm_output.squeeze(), sarcasm_labels)
            
            acc_bully = accuracy_bully(bully_output, bully_labels)
            f1_bully_metric = f1_bully(bully_output, bully_labels)
            
            acc_harm = accuracy_harmful_score(harmful_score_output, harmful_score_labels)
            f1_harm = f1_harmful_score(harmful_score_output, harmful_score_labels)
            
            acc_target = accuracy_target(target_output, target_labels)
            f1_target_metric = f1_target(target_output, target_labels)
            
            #tqdm.write(f" Training Batch Loss: {loss.item():.4f}")
            # Update progress bar with current batch loss
            #batch_iterator.set_postfix({'Train Batch Loss': loss.item(), 'Training Avg Loss': running_loss / (batch_idx + 1)})
            #tqdm.set_postfix(batch_loss=loss.item(), avg_loss=running_loss / (len(data_sample) + 1))
            # Update tqdm progress bar with current batch loss and metrics
            batch_iterator.set_postfix({
                'Train Loss': loss.item(),
                'Acc Sent': acc_sent.item(),
                'Acc Emo': acc_emo.item(),
                'Acc Sarc': acc_sarc.item(),
                'Acc Bully': acc_bully.item(),
                'Acc Harm': acc_harm.item(),
                'Acc Target': acc_target.item(),
                'F1 Sent': f1_sent.item(),
                'F1 Emo': f1_emo.item(),
                'F1 Sarc': f1_sarc.item(),
                'F1 Bully': f1_bully_metric.item(),
                'F1 Harm': f1_harm.item(),
                'F1 Target': f1_target_metric.item()
            })

        # Print the average loss for this epoch
        epoch_loss = running_loss / len(train_dataloader)
        losses.append(epoch_loss)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        
        # Validation
        model.eval()
        val_running_loss = 0.0
    
        with torch.no_grad():
            # batch_iterator = tqdm(val_dataloader, desc=f"Processing Validation {epoch:02d}")
            # Initialize accumulators for losses and metrics
            val_loss_sum = 0.0
            acc_sent_sum, f1_sent_sum = 0.0, 0.0
            acc_emo_sum, f1_emo_sum = 0.0, 0.0
            acc_sarc_sum, f1_sarc_sum = 0.0, 0.0
            acc_bully_sum, f1_bully_sum = 0.0, 0.0
            acc_harm_sum, f1_harm_sum = 0.0, 0.0
            acc_target_sum, f1_target_sum = 0.0, 0.0
            
            accuracy_sentiment.reset()
            f1_sentiment.reset()
            accuracy_emotion.reset()
            f1_emotion.reset()
            accuracy_sarcasm.reset()
            f1_sarcasm.reset()
            accuracy_bully.reset()
            f1_bully.reset()
            accuracy_harmful_score.reset()
            f1_harmful_score.reset()
            accuracy_target.reset()
            f1_target.reset()
            
            batch_iterator = tqdm(enumerate(val_dataloader), total=len(train_dataloader), desc=f'Epoch [{epoch+1}/{num_epochs}]')
            for batch_idx, (images, input_ids, attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels) in batch_iterator:
                #extract features
                images = images.to(device)
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                sentiment_labels = sentiment_labels.to(device)
                emotion_labels = emotion_labels.to(device)
                sarcasm_labels = sarcasm_labels.to(device)
                bully_labels = bully_labels.to(device)
                harmful_score_labels = harmful_score_labels.to(device)
                target_labels = target_labels.to(device)
                
                
                # Forward pass
                sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output = model(images, input_ids, attention_mask)

                # Calculate Loss
                # Compute the losses

                loss_sentiment = loss_fn_sentiment(sentiment_output, sentiment_labels)
                loss_emotion = loss_fn_emotion(emotion_output, emotion_labels)
                loss_sarcasm = loss_fn_sarcasm(sarcasm_output.squeeze(), sarcasm_labels.float())
                loss_bully = loss_fn_bully(bully_output, bully_labels)
                loss_harmful_score = loss_fn_harmful_score(harmful_score_output, harmful_score_labels)
                loss_target = loss_fn_target(target_output, target_labels)
                
                # Combine the losses
                loss = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target
                
                # Update running loss
                val_running_loss += loss.item()
                
                # Calculate metrics
                val_acc_sent = accuracy_sentiment(sentiment_output, sentiment_labels)
                val_f1_sent = f1_sentiment(sentiment_output, sentiment_labels)
                
                val_acc_emo = accuracy_emotion(emotion_output, emotion_labels)
                val_f1_emo = f1_emotion(emotion_output, emotion_labels)
                
                val_acc_sarc = accuracy_sarcasm(sarcasm_output.squeeze(), sarcasm_labels)
                val_f1_sarc = f1_sarcasm(sarcasm_output.squeeze(), sarcasm_labels)
                
                val_acc_bully = accuracy_bully(bully_output, bully_labels)
                val_f1_bully_metric = f1_bully(bully_output, bully_labels)
                
                val_acc_harm = accuracy_harmful_score(harmful_score_output, harmful_score_labels)
                val_f1_harm = f1_harmful_score(harmful_score_output, harmful_score_labels)
                
                val_acc_target = accuracy_target(target_output, target_labels)
                val_f1_target_metric = f1_target(target_output, target_labels)
                
                # Accumulate the metrics
                acc_sent_sum += val_acc_sent.item()
                f1_sent_sum += val_f1_sent.item()

                acc_emo_sum += val_acc_emo.item()
                f1_emo_sum += val_f1_emo.item()

                acc_sarc_sum += val_acc_sarc.item()
                f1_sarc_sum += val_f1_sarc.item()

                acc_bully_sum += val_acc_bully.item()
                f1_bully_sum += val_f1_bully_metric.item()

                acc_harm_sum += val_acc_harm.item()
                f1_harm_sum += val_f1_harm.item()

                acc_target_sum += val_acc_target.item()
                f1_target_sum += val_f1_target_metric.item()
                
                #tqdm.write(f" Validation Batch Loss: {loss.item():.4f}")
                #batch_iterator.set_postfix({'Val Batch Loss': loss.item(), 'Val Avg Loss': val_running_loss / (batch_idx + 1)})
                #tqdm.set_postfix(batch_loss=loss.item(), avg_loss=val_running_loss / (len(data_sample) + 1))
                
                # Update tqdm progress bar with validation metrics
                batch_iterator.set_postfix({
                    'Val Loss': loss.item(),
                    'Val Acc Sent': val_acc_sent.item(),
                    'Val Acc Emo': val_acc_emo.item(),
                    'Val Acc Sarc': val_acc_sarc.item(),
                    'Val Acc Bully': val_acc_bully.item(),
                    'Val Acc Harm': val_acc_harm.item(),
                    'Val Acc Target': val_acc_target.item(),
                    'Val F1 Sent': val_f1_sent.item(),
                    'Val F1 Emo': val_f1_emo.item(),
                    'Val F1 Sarc': val_f1_sarc.item(),
                    'Val F1 Bully': val_f1_bully_metric.item(),
                    'Val F1 Harm': val_f1_harm.item(),
                    'Val F1 Target': val_f1_target_metric.item()
                })
                
            # Print the average loss for this epoch
            val_running_loss = val_running_loss / len(val_dataloader)
            vals.append(val_running_loss)
            print(f'Epoch {epoch+1}/{num_epochs}, Val_Loss: {val_running_loss:.4f}')
            
            # Calculate averages for losses and metrics
            num_batches = len(val_dataloader)
            avg_val_loss = val_loss_sum / num_batches
            avg_acc_sent = acc_sent_sum / num_batches
            avg_f1_sent = f1_sent_sum / num_batches

            avg_acc_emo = acc_emo_sum / num_batches
            avg_f1_emo = f1_emo_sum / num_batches

            avg_acc_sarc = acc_sarc_sum / num_batches
            avg_f1_sarc = f1_sarc_sum / num_batches

            avg_acc_bully = acc_bully_sum / num_batches
            avg_f1_bully = f1_bully_sum / num_batches

            avg_acc_harm = acc_harm_sum / num_batches
            avg_f1_harm = f1_harm_sum / num_batches

            avg_acc_target = acc_target_sum / num_batches
            avg_f1_target = f1_target_sum / num_batches

            # Log the averaged metrics
            print(f'Validation Loss: {avg_val_loss:.4f}')
            print(f'Acc Sent: {avg_acc_sent:.4f}, F1 Sent: {avg_f1_sent:.4f}')
            print(f'Acc Emo: {avg_acc_emo:.4f}, F1 Emo: {avg_f1_emo:.4f}')
            print(f'Acc Sarc: {avg_acc_sarc:.4f}, F1 Sarc: {avg_f1_sarc:.4f}')
            print(f'Acc Bully: {avg_acc_bully:.4f}, F1 Bully: {avg_f1_bully:.4f}')
            print(f'Acc Harm: {avg_acc_harm:.4f}, F1 Harm: {avg_f1_harm:.4f}')
            print(f'Acc Target: {avg_acc_target:.4f}, F1 Target: {avg_f1_target:.4f}')
            

    print('Training complete')



In [42]:
# Trainavg_val_loss = val_loss_sum / len(val_dataloader)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
losses = []
vals = []
train_model(model, train_dataloader, val_dataloader, optimizer, losses, vals, num_epochs=25)

Epoch [1/5]: 100%|██████████| 154/154 [00:52<00:00,  2.93it/s, Train Loss=0.57, Acc Sent=1, Acc Emo=0.5, Acc Sarc=1, Acc Bully=1, Acc Harm=1, Acc Target=0.667, F1 Sent=1, F1 Emo=0.524, F1 Sarc=1, F1 Bully=1, F1 Harm=1, F1 Target=0.7]                            


Epoch 1/5, Loss: 0.6598


Epoch [1/5]: 100%|██████████| 154/154 [00:20<00:00,  7.38it/s, Val Loss=0.0367, Val Acc Sent=1, Val Acc Emo=1, Val Acc Sarc=1, Val Acc Bully=1, Val Acc Harm=1, Val Acc Target=1, Val F1 Sent=1, Val F1 Emo=1, Val F1 Sarc=1, Val F1 Bully=1, Val F1 Harm=1, Val F1 Target=1]                                    


Epoch 1/5, Val_Loss: 1.3094


Epoch [2/5]: 100%|██████████| 154/154 [00:52<00:00,  2.93it/s, Train Loss=0.523, Acc Sent=0.9, Acc Emo=0.958, Acc Sarc=1, Acc Bully=1, Acc Harm=1, Acc Target=1, F1 Sent=0.918, F1 Emo=0.958, F1 Sarc=1, F1 Bully=1, F1 Harm=1, F1 Target=1]                               


...

Epoch 4/5, Val_Loss: 1.1671


Epoch [5/5]: 100%|██████████| 154/154 [00:52<00:00,  2.92it/s, Train Loss=0.407, Acc Sent=1, Acc Emo=0.75, Acc Sarc=0.929, Acc Bully=1, Acc Harm=1, Acc Target=0.464, F1 Sent=1, F1 Emo=0.75, F1 Sarc=0.933, F1 Bully=1, F1 Harm=1, F1 Target=0.481]                      


Epoch 5/5, Loss: 0.5260


Epoch [5/5]: 100%|██████████| 154/154 [00:21<00:00,  7.28it/s, Val Loss=0.0439, Val Acc Sent=1, Val Acc Emo=1, Val Acc Sarc=1, Val Acc Bully=1, Val Acc Harm=1, Val Acc Target=1, Val F1 Sent=1, Val F1 Emo=1, Val F1 Sarc=1, Val F1 Bully=1, Val F1 Harm=1, Val F1 Target=1]                                    

Epoch 5/5, Val_Loss: 1.3024
Training complete


In [60]:
train_model(model, train_dataloader, val_dataloader, optimizer, losses, vals, num_epochs=1)

Epoch [1/1]: 100%|██████████| 154/154 [00:52<00:00,  2.93it/s, Train Loss=0.812, Acc Sent=1, Acc Emo=0.833, Acc Sarc=0.857, Acc Bully=0.875, Acc Harm=0.464, Acc Target=1, F1 Sent=1, F1 Emo=0.85, F1 Sarc=0.9, F1 Bully=0.925, F1 Harm=0.963, F1 Target=1]          


Epoch 1/1, Loss: 0.4935


Epoch [1/1]:  25%|██▌       | 39/154 [00:05<00:15,  7.41it/s, Val Loss=14.7, Val Acc Sent=0.417, Val Acc Emo=0.208, Val Acc Sarc=0.625, Val Acc Bully=0.375, Val Acc Harm=1, Val Acc Target=0.429, Val F1 Sent=0.577, Val F1 Emo=0.229, Val F1 Sarc=0.571, Val F1 Bully=0.857, Val F1 Harm=1, Val F1 Target=0.75]       

Epoch 1/1, Val_Loss: 10.7649
Validation Loss: 0.0000
Acc Sent: 0.5438, F1 Sent: 0.6723
Acc Emo: 0.1539, F1 Emo: 0.2223
Acc Sarc: 0.5609, F1 Sarc: 0.5259
Acc Bully: 0.6020, F1 Bully: 0.7168
Acc Harm: 0.9060, F1 Harm: 0.9787
Acc Target: 0.3822, F1 Target: 0.7033
Training complete


In [29]:
! pip install -q torchviz

In [32]:
from torchviz import make_dot

make_dot((sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output), params=dict(list(model.named_parameters()))).render("MultiTaskMeme_torchviz", format="png")

'MultiTaskMeme_torchviz.png'

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [56]:
# Validation phase
model.eval()  # Set model to evaluation mode
total_val_loss = 0
epoch = 27
# Initialize lists to store true and predicted labels for each task
all_labels_bully = []
all_preds_bully = []

all_labels_sentiment = []
all_preds_sentiment = []

all_labels_emotion = []
all_preds_emotion = []

all_labels_sarcasm = []
all_preds_sarcasm = []

all_labels_harmful_score = []
all_preds_harmful_score = []

all_labels_target = []
all_preds_target = []

with torch.no_grad():  # Disable gradient calculation
    for images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels in val_dataloader:
        # Move data to the GPU
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)
        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)

        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out, harmful_score_out, target_out = model(images, text_input_ids, text_attention_mask)

        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)

        # Total loss
        total_val_loss += (loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target).item()

        # Get predictions for each task
        _, predicted_sentiment = torch.max(sentiment_out, 1)
        _, predicted_emotion = torch.max(emotion_out, 1)
        _, predicted_sarcasm = torch.max(sarcasm_out, 1)
        _, predicted_bully = torch.max(bully_out, 1)
        _, predicted_harmful_score = torch.max(harmful_score_out, 1)  # Assuming multi-class
        _, predicted_target = torch.max(target_out, 1)  # Assuming multi-class

        # Collect true and predicted labels for each task
        all_labels_sentiment.append(sentiment_labels.cpu().numpy())
        all_preds_sentiment.append(predicted_sentiment.cpu().numpy())

        all_labels_emotion.append(emotion_labels.cpu().numpy())
        all_preds_emotion.append(predicted_emotion.cpu().numpy())

        all_labels_sarcasm.append(sarcasm_labels.cpu().numpy())
        all_preds_sarcasm.append(predicted_sarcasm.cpu().numpy())

        all_labels_bully.appenavg_val_loss = val_loss_sum / len(val_dataloader)d(bully_labels.cpu().numpy())
        all_preds_bully.append(predicted_bully.cpu().numpy())

        all_labels_harmful_score.append(harmful_score_labels.cpu().numpy())
        all_preds_harmful_score.append(predicted_harmful_score.cpu().numpy())

        all_labels_target.append(target_labels.cpu().numpy())
        all_preds_target.append(predicted_target.cpu().numpy())

avg_val_loss = total_val_loss / len(val_dataloader)

# Flatten lists for each task
all_labels_bully = np.concatenate(all_labels_bully)
all_preds_bully = np.concatenate(all_preds_bully)

all_labels_sentiment = np.concatenate(all_labels_sentiment)
all_preds_sentiment = np.concatenate(all_preds_sentiment)

all_labels_emotion = np.concatenate(all_labels_emotion)
all_preds_emotion = np.concatenate(all_preds_emotion)

all_labels_sarcasm = np.concatenate(all_labels_sarcasm)
all_preds_sarcasm = np.concatenate(all_preds_sarcasm)

all_labels_harmful_score = np.concatenate(all_labels_harmful_score)
all_preds_harmful_score = np.concatenate(all_preds_harmful_score)

all_labels_target = np.concatenate(all_labels_target)
all_preds_target = np.concatenate(all_preds_target)

# Calculate accuracy and F1 score for each task
accuracy_bully = accuracy_score(all_labels_bully, all_preds_bully)
f1_bully = f1_score(all_labels_bully, all_preds_bully, average='weighted')

accuracy_sentiment = accuracy_score(all_labels_sentiment, all_preds_sentiment)
f1_sentiment = f1_score(all_labels_sentiment, all_preds_sentiment, average='weighted')

accuracy_emotion = accuracy_score(all_labels_emotion, all_preds_emotion)
f1_emotion = f1_score(all_labels_emotion, all_preds_emotion, average='weighted')

accuracy_sarcasm = accuracy_score(all_labels_sarcasm, all_preds_sarcasm)
f1_sarcasm = f1_score(all_labels_sarcasm, all_preds_sarcasm, average='weighted')

accuracy_harmful_score = accuracy_score(all_labels_harmful_score, all_preds_harmful_score)
f1_harmful_score = f1_score(all_labels_harmful_score, all_preds_harmful_score, average='weighted')

accuracy_target = accuracy_score(all_labels_target, all_preds_target)
f1_target = f1_score(all_labels_target, all_preds_target, average='weighted')

print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f},\n'
      f'Bully Accuracy: {accuracy_bully:.4f}, F1 Score: {f1_bully:.4f},\n'
      f'Sentiment Accuracy: {accuracy_sentiment:.4f}, F1 Score: {f1_sentiment:.4f},\n'
      f'Emotion Accuracy: {accuracy_emotion:.4f}, F1 Score: {f1_emotion:.4f},\n'
      f'Sarcasm Accuracy: {accuracy_sarcasm:.4f}, F1 Score: {f1_sarcasm:.4f},\n'
      f'Harmful Score Accuracy: {accuracy_harmful_score:.4f}, F1 Score: {f1_harmful_score:.4f},\n'
      f'Target Accuracy: {accuracy_target:.4f}, F1 Score: {f1_target:.4f}')


Epoch 27, Validation Loss: 11.0457,
Bully Accuracy: 0.7321, F1 Score: 0.7105,
Sentiment Accuracy: 0.6558, F1 Score: 0.6632,
Emotion Accuracy: 0.2419, F1 Score: 0.2367,
Sarcasm Accuracy: 0.5016, F1 Score: 0.3351,
Harmful Score Accuracy: 0.9838, F1 Score: 0.9786,
Target Accuracy: 0.7062, F1 Score: 0.6969


In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the dataset and dataloader
dataset = MemeDataset(df, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)  # Adjust batch size as needed

# Initialize the model and move it to the GPU
model = MultiTaskModel()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)  # Use DataParallel if multiple GPUs are available
model.to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Loss functions
loss_fn_sentiment = nn.CrossEntropyLoss().to(device)
loss_fn_emotion = nn.CrossEntropyLoss().to(device)
loss_fn_sarcasm = nn.BCEWithLogitsLoss().to(device)
loss_fn_bully = nn.CrossEntropyLoss().to(device)
loss_fn_harmful_score = nn.CrossEntropyLoss().to(device)
loss_fn_target = nn.CrossEntropyLoss().to(device)

# Training loop
for epoch in range(10):  # Set epochs accordingly
    model.train()
    
    total_loss = 0  # Initialize total loss for the epoch
    
#     for images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels in train_dataloader:
        # Move data to the GPU
    for images, text_input_ids, text_attention_mask, sentiment_labels, emotion_labels, sarcasm_labels, bully_labels, harmful_score_labels, target_labels in train_dataloader:
  
        images = images.to(device)
        text_input_ids = text_input_ids.to(device)
        text_attention_mask = text_attention_mask.to(device)
        sentiment_labels = sentiment_labels.to(device)
        emotion_labels = emotion_labels.to(device)
        sarcasm_labels = sarcasm_labels.to(device)xpected 4D input (got 3D input)

        bully_labels = bully_labels.to(device)
        harmful_score_labels = harmful_score_labels.to(device)
        target_labels = target_labels.to(device)
        
        optimizer.zero_grad()  # Clear gradients at the start of each batch
        
        # Forward pass
        sentiment_out, emotion_out, sarcasm_out, bully_out = model(images, text_input_ids, text_attention_mask)
        
        # Compute loss for each task
        loss_sentiment = loss_fn_sentiment(sentiment_out, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_out, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_out.squeeze(), sarcasm_labels.float())  # Squeeze if necessary
        loss_bully = loss_fn_bully(bully_out, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_out, harmful_score_labels)
        loss_target = loss_fn_target(target_out, target_labels)
        
        # Total loss (sum or weigh the losses as needed)
        total_loss_batch = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target
        
        # Backward pass and optimization
        total_loss_batch.backward()
        optimizer.step()  # Update model parameters
        
        total_loss += total_loss_batch.item()  # Accumulate loss for the epoch

    # Optionally clear cache at the end of each epoch
    torch.cuda.empty_cache()  
    
    # Print the average loss for the epoch
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')


# Multi Bully using CLIP Model Implementation

In [10]:
MODELS = {
     "RN50": "https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt",
     "RN101": "https://openaipublic.azureedge.net/clip/models/8fa8567bab74a42d41c5915025a8e4538c3bdbe8804a470a72f30b0d94fab599/RN101.pt",
     "RN50x4": "https://openaipublic.azureedge.net/clip/models/7e526bd135e493cef0776de27d5f42653e6b4c8bf9e0f653bb11773263205fdd/RN50x4.pt",
     "ViT-B/32": "https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt"
 }

In [11]:
!wget {MODELS["ViT-B/32"]} -O clip_model.pt

--2024-11-24 13:59:15--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.70, 2620:1ec:29:1::70
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353976522 (338M) [application/octet-stream]
Saving to: 'clip_model.pt'

clip_model.pt       100%[===================>] 337.58M  65.9MB/s    in 6.0s    

2024-11-24 13:59:21 (56.2 MB/s) - 'clip_model.pt' saved [353976522/353976522]



In [13]:
clip_model = torch.jit.load("clip_model.pt").cuda().eval()
input_resolution = clip_model.input_resolution.item()
context_length = clip_model.context_length.item()
vocab_size = clip_model.vocab_size.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in clip_model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [12]:
from torchvision import transforms, models
from transformers import RobertaTokenizer, RobertaModel

# Load CLIP, VGG19, and Roberta models
clip_model = torch.jit.load("clip_model.pt").cuda().eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load VGG19 Model
vgg19 = models.vgg19(pretrained=True).features.to(device).eval()
resnet18 = models.resnet50(weights="IMAGENET1K_V1").to(device).eval()

# Load RoBERTa TOkenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaModel.from_pretrained('roberta-base').to(device)

# Image transformation for VGG19 and CLIP
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 212MB/s] 
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 195MB/s]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
clip_model

In [ ]:
# ! pip install -q ftfy regex tqdm
# ! pip install -q git+https://github.com/openai/CLIP.git

In [ ]:
# import torch
# import clip
# from PIL import Image

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)
# xpected 4D input (got 3D input)

# image = preprocess(Image.open("/kaggle/working/bully_data/data/bully_data/45.jpg")).unsqueeze(0).to(device)
# text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

# with torch.no_grad():
#     image_features = model.encode_image(image)
#     text_features = model.encode_text(text)
    
#     logits_per_image, logits_per_text = model(image, text)
#     probs = logits_per_image.softmax(dim=-1).cpu().numpy()

# print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

In [ ]:
# clip.tokenize(["RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized"]).to(device)

In [13]:
class MemeDatasetClipVGG(Dataset):
    def __init__(self, dataframe, transform, clip_model, vgg19):
        self.dataframe = dataframe
        self.transform = transform
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.roberta_model = RobertaModel.from_pretrained('roberta-base').to(device)
        self.clip_model = clip_model
        self.vgg19 = vgg19
        #self.resnet = resnet50
        
        self.img_folder = '/kaggle/working/bully_data/data/bully_data'
        
        # Define label mappings
        self.text_label_mapping = {
            "Bully": 1,
            "Nonbully": 0
        }
        
        self.sentiment_mapping = {
            "Positive":1,
            "Neutral": 0,
            "Negative": 2
        }
        
        self.emotion_mapping = {
            "Disgust": 0,
            "Ridicule": 1,
            "Sadness": 2,
            "Surprise": 3,
            "Anticipation": 4,
            "Angry": 5,
            "Happiness": 6,
            "Other": 7,
            "Trust": 8,
            "Fear": 9
        }
        
        self.sarcasm_mapping = {
            "Yes": 1,
            "No": 0
        }
        
        self.harmful_score_mapping = {
            "Harmless": 0,
            "Partially-Harmful": 1,
            "Very-Harmful": 2
        }
        
        self.target_mapping = {
            "Individual": 0,
            "Society": 1,
            "Organization": 2,
            "Community": 3
        }
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        #print(f"<*>Loading {idx} th Data point-----")
        # Load image
        img_name = self.dataframe.iloc[idx]['Img_Name']
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        
        img_tensor = torch.tensor(image.unsqueeze(0)).to(device)
        #img_tensor = self.transform(image).unsqueeze(0).to(device)  # Transform and add batch dimension
    
        
        # 2. Extract image features using CLIP and VGG19
        with torch.no_grad():
            #print("Loading CLIP model")
            image_clip_input = self.clip_model.encode_image(img_tensor).float().to(device)  #(batch_size, 512)
            #print("Loading VGG19 model")
            image_vgg_feature = self.vgg19(img_tensor).view(-1).float().to(device)  # Flattened VGG19 features (batch_size, 1024)
            #print("|------VGG Features Done")
            
        # 3. Process text with CLIP and RoBERTa
        text = self.dataframe.iloc[idx]['Img_Text']
        text = text.replace("\n", "")
        #print(f" Datapoint {idx}: Img_Text: {text}  ")
        #print("|------IMG_Text Loaded Success")
        tokens = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=77)
        #print("|------Tokenizer Success")
        input_ids = tokens['input_ids'].to(device)
        #print("|------Input IDs Success")
        attention_mask = tokens['attention_mask'].to(device)
        #print("|------Attention Mask Success")
        
        #clip_tokens = clip.tokenize([text]).to(device)
        

        with torch.no_grad():
            #print("Loading CLIP text Encoding......")
            #print(input_ids)
            text_clip_input = self.clip_model.encode_text(input_ids).float().to(device)  #(batch_size, 512)
            #print("    |------CLIP Text input Encoding Success")
            text_roberta_output = self.roberta_model(input_ids, attention_mask=attention_mask)
            #print("    |------RoBERTa output with attention Mask Success")
            text_roberta_embedding = text_roberta_output.last_hidden_state[:, 0, :].float().to(device)  #(batch_size, 768)
            #print("|------Roberta embedding Success")
        
        # Load and tokenize text
        #text = self.dataframe.iloc[idx]['Img_Text']
        #inputs = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        
        # Get labels and apply mappings
        sentiment_label = torch.tensor(self.sentiment_mapping[self.dataframe.iloc[idx]['Sentiment']], dtype=torch.long)
        emotion_label = torch.tensor(self.emotion_mapping[self.dataframe.iloc[idx]['Emotion']], dtype=torch.long)
        sarcasm_label = torch.tensor(self.sarcasm_mapping[self.dataframe.iloc[idx]['Sarcasm']], dtype=torch.float)  # Binary sarcasm
        bully_label = torch.tensor(self.text_label_mapping[self.dataframe.iloc[idx]['Img_Label']], dtype=torch.long)  # Bully detection
        harmful_score_label = torch.tensor(self.harmful_score_mapping[self.dataframe.iloc[idx]['Harmful_Score']], dtype=torch.long)
        target_label = torch.tensor(self.target_mapping[self.dataframe.iloc[idx]['Target']], dtype=torch.long)
        #print("|------Processed------|")
        # 5. Prepare the sample dictionary
        sample = {
            "id": idx,
            "image": image, # 3x224x224
            "image_clip_input": image_clip_input,  # CLIP image features (1x512-dim)
            "image_vgg_feature": image_vgg_feature,  # VGG19 image features (flattened 1x25088-dim)
            "text_clip_input": text_clip_input,  # CLIP text features (1x512-dim)
            "text_roberta_embedding": text_roberta_embedding,  # RoBERTa text embedding (1x768-dim)
            "bully_label": bully_label,  # Cyberbullying label
            "sentiment": sentiment_label,  # Sentiment label
            "sarcasm": sarcasm_label,  # Sarcasm label
            "emotion": emotion_label,  # Emotion label
            "harmful-score": harmful_score_label,  # Harmful score
            "target": target_label  # Target variable
        }
        
        #print("|------Sampled------|")
        
        return sample

In [14]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
# os.environ["TORCH_USE_CUDA_DSA"] = "0"
torch.cuda.empty_cache()

In [16]:
# Sanity Check for Datasets

import os

# Load the dataset (assume the dataframe and image directory are available)
dataset_clip_vgg = MemeDatasetClipVGG(df, transform, clip_model, vgg19)

# Create a DataLoader for batching
dataloader_clip_vgg = DataLoader(dataset_clip_vgg, batch_size=16, shuffle=False)
batch_iterator_dataloader_vgg = tqdm(dataloader_clip_vgg, desc=f"Processing DataLoader")
# Iterate through the dataloader
sample = None
for batch in dataloader_clip_vgg:
    ##print(f"Len: {len(batch)}") 
    # Further model training steps...
    #torch.cuda.empty_cache()
    sample=batch
    break
    
print("Done")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing DataLoader:   0%|          | 0/193 [00:00<?, ?it/s]/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(image.unsqueeze(0)).to(device)


Done


In [17]:
print(sample.keys())
for k, v in sample.items():
    print(f"Key: {k};  --> shape: {v.shape}")

dict_keys(['id', 'image', 'image_clip_input', 'image_vgg_feature', 'text_clip_input', 'text_roberta_embedding', 'bully_label', 'sentiment', 'sarcasm', 'emotion', 'harmful-score', 'target'])
Key: id;  --> shape: torch.Size([16])
Key: image;  --> shape: torch.Size([16, 3, 224, 224])
Key: image_clip_input;  --> shape: torch.Size([16, 1, 512])
Key: image_vgg_feature;  --> shape: torch.Size([16, 25088])
Key: text_clip_input;  --> shape: torch.Size([16, 1, 512])
Key: text_roberta_embedding;  --> shape: torch.Size([16, 1, 768])
Key: bully_label;  --> shape: torch.Size([16])
Key: sentiment;  --> shape: torch.Size([16])
Key: sarcasm;  --> shape: torch.Size([16])
Key: emotion;  --> shape: torch.Size([16])
Key: harmful-score;  --> shape: torch.Size([16])
Key: target;  --> shape: torch.Size([16])


In [15]:
class ImageClipModel(nn.Module):
    def __init__(self,input_dim):
        super(ImageClipModel, self).__init__()
        self.Linear_input = nn.Linear(input_dim, 512)
        self.Linear_512_256 = nn.Linear(512, 256)
        self.Linear_256_128 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.2)
        self.leakyReLU = nn.LeakyReLU(0.01)
        self.ReLU = nn.ReLU()
        
        
    def forward(self, x):
        x = self.Linear_input(x)
        x = self.leakyReLU(x)
        x = self.Linear_512_256(x)
        x = self.leakyReLU(x)
        x = self.Linear_256_128(x)
        return x
        

In [19]:
image_clip_model = ImageClipModel(512).to(device)
data_sample = dataset_clip_vgg[0]
image_clip_data = data_sample["image_clip_input"]
print(image_clip_data.shape)
y = image_clip_model(image_clip_data)
y.shape

torch.Size([1, 512])


/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(image.unsqueeze(0)).to(device)


torch.Size([1, 128])

In [16]:
class ImageVGGDenseModel(nn.Module):
    def __init__(self, input_dim):
        super(ImageVGGDenseModel, self).__init__()
        self.Linear_input = nn.Linear(input_dim, 25088) # input_dim: 25088
        self.Linear_25088_1024 = nn.Linear(25088, 1024)
        self.Linear_1024_512 = nn.Linear(1024, 512)
        self.Linear_512_256 = nn.Linear(512, 256)
        self.Linear_256_128 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.2)
        self.leakyReLU = nn.LeakyReLU(0.01)
        self.ReLU = nn.ReLU()
        
        
    def forward(self, x):
        x = self.Linear_input(x)
        x = self.leakyReLU(x)
        x = self.Linear_25088_1024(x)
        x = self.leakyReLU(x)
        x = self.Linear_1024_512(x)
        return x
        

In [21]:
image_vgg_model = ImageVGGDenseModel(25088).to(device)
data_sample = dataset_clip_vgg[0]
image_vgg_data = data_sample["image_vgg_feature"]
print(image_vgg_data.shape)
y = image_vgg_model(image_vgg_data)
y.shape

torch.Size([25088])


/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(image.unsqueeze(0)).to(device)


torch.Size([512])

In [17]:
class TextClipModel(nn.Module):
    def __init__(self, input_dim):
        super(TextClipModel, self).__init__()
        self.Linear_input = nn.Linear(input_dim, 512)
        self.Linear_512_256 = nn.Linear(512, 256)
        self.Linear_256_128 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.2)
        self.leakyReLU = nn.LeakyReLU(0.01)
        self.ReLU = nn.ReLU()
        
        
    def forward(self, x):
        x = self.Linear_input(x)
        x = self.leakyReLU(x)
        x = self.Linear_512_256(x)
        x = self.leakyReLU(x)
        x = self.Linear_256_128(x)
        return x

In [23]:
text_clip_model = TextClipModel(512).to(device)
data_sample = dataset_clip_vgg[0]
text_clip_data = data_sample["text_clip_input"]
print(text_clip_data.shape)
y = text_clip_model(text_clip_data)
y.shape

torch.Size([1, 512])


/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(image.unsqueeze(0)).to(device)


torch.Size([1, 128])

In [18]:
class TextRobertaModel(nn.Module):
    def __init__(self, input_dim):
        super(TextRobertaModel, self).__init__()
        self.Linear_input = nn.Linear(input_dim, 768)
        self.Linear_768_512 = nn.Linear(768, 512)
        self.Linear_512_128 = nn.Linear(512, 128)
        self.Linear_512_256 = nn.Linear(512, 256)
        self.Linear_256_128 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.2)
        self.leakyReLU = nn.LeakyReLU(0.01)
        self.ReLU = nn.ReLU()
        
        
    def forward(self, x):
        x = self.Linear_input(x)
        x = self.leakyReLU(x)
        x = self.Linear_768_512(x)
        x = self.leakyReLU(x)
        x = self.Linear_512_128(x)
#         x = self.leakyReLU(x)
#         x = self.Linear_256_128(x)
        return x

In [25]:
text_roberta_model = TextRobertaModel(768).to(device)
data_sample = dataset_clip_vgg[0]
text_roberta_data = data_sample["text_roberta_embedding"]
print(text_roberta_data.shape)
y = text_roberta_model(text_roberta_data)
y.shape

torch.Size([1, 768])


/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(image.unsqueeze(0)).to(device)


torch.Size([1, 128])

In [26]:
# class ResNetPreModel(nn.Module):
#     def __init__(self):
#         super(ResNetPreModel, self).__init__()
#         # Load pre-trained ResNet model for image processing
#         self.image_model = models.resnet18(pretrained=True)
#         self.image_model.fc = nn.Linear(self.image_model.fc.in_features, 512)  # Modify the final layer
        
#     def forward(self, image):
#         x = self.image_model(image)
#         return x

In [27]:
# resnet_model = ResNetPreModel()
# data_sample = dataset_clip_vgg[0]
# image = data_sample["image"].unsqueeze(0)
# print(image.shape)
# y = resnet_model(image)
# y.shape

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth

  0%|          | 0.00/44.7M [00:00<?, ?B/s]
 17%|█▋        | 7.50M/44.7M [00:00<00:00, 77.5MB/s]
 36%|███▌      | 15.9M/44.7M [00:00<00:00, 83.4MB/s]
 55%|█████▍    | 24.5M/44.7M [00:00<00:00, 86.4MB/s]
 73%|███████▎  | 32.8M/44.7M [00:00<00:00, 80.4MB/s]
100%|██████████| 44.7M/44.7M [00:00<00:00, 83.8MB/s]
/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c

torch.Size([1, 3, 224, 224])


torch.Size([1, 512])

In [19]:
class SelfAttentionBiLSTM(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttentionBiLSTM, self).__init__()
        
        self.bilstm = nn.LSTM(input_size=input_dim, hidden_size=input_dim, bidirectional=True, batch_first=True)
        self.fc_q = nn.Linear(1536, 512)
        self.fc_k = nn.Linear(1536, 512)
        self.fc_v = nn.Linear(1536, 512)
        
    def attention_fusion(self, vec1, vec2):
        img_text = torch.cat((vec1, vec2), 1)
        prob_img = torch.sigmoid(self.prob_img(img_text))
        prob_txt = torch.sigmoid(self.prob_text(img_text))
        
        vec1 = prob_img * vec1
        vec2 = prob_txt * vec2
        
        out_rep = torch.cat((vec1, vec2), 1)
        
        return out_rep 
    
    def forward(self, x):
        # Ensure input has batch, sequence, and feature dimensions
        if len(x.shape) == 2:
            x = x.unsqueeze(1)  # Add a sequence length dimension for single-time-step inputs
        #print(f"Input Shape after unsqueeze: {x.shape}")

        batch_size = x.shape[0]
        #print(f"Batch Size: {batch_size}")
        
        # Initialize h0 and c0 for LSTM with batch size
        h0 = torch.randn(2 * 1, batch_size, 768).to(x.device)  # (num_layers * num_directions, batch_size, hidden_size)
        c0 = torch.randn(2 * 1, batch_size, 768).to(x.device)  # (num_layers * num_directions, batch_size, hidden_size)
                
        after_lstm = self.bilstm(x, (h0, c0))[0]

        q = F.relu(self.fc_q(after_lstm))
        k = F.relu(self.fc_k(after_lstm))
        v = F.relu(self.fc_v(after_lstm))

        att = F.tanh(torch.bmm(q, k.transpose(1, 2)))  # Transpose k for batch matmul
        #print("Attention Shape :", att.shape)
        
        soft = F.softmax(att, dim=-1)
        #print("Softmax Shape :", soft.shape)
        
        value = torch.mean(torch.bmm(soft, v), dim=1)
        #print("Value Shape :", value.shape)
        
        return value

In [29]:
self_attention_model = SelfAttentionBiLSTM(768).to(device)
data_sample = dataset_clip_vgg[0]
text_roberta_data = data_sample["text_roberta_embedding"]
print(text_roberta_data.shape)
y = self_attention_model(text_roberta_data)
y.shape

torch.Size([1, 768])


/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(image.unsqueeze(0)).to(device)


torch.Size([1, 512])

In [20]:
class MemeModelCLIPVGG(nn.Module):
    def __init__(self,img_clip_inp_dim, vgg_inp_dim, text_clip_inp_dim, text_roberta_inp_dim):
        super(MemeModelCLIPVGG,self).__init__()
        
        self.image_clip_model = ImageClipModel(img_clip_inp_dim)
        self.vgg_dense =  ImageVGGDenseModel(vgg_inp_dim)
        self.text_clip_model = TextClipModel(text_clip_inp_dim)
        self.text_roberta_model = TextRobertaModel(text_roberta_inp_dim)
        #self.resnet_model = ResNetPreModel()
        
        self.attention_bilstm_model = SelfAttentionBiLSTM(text_roberta_inp_dim)
        
        # Load pre-trained RoBERTa model
        #self.text_roberta_model = RobertaModel.from_pretrained('roberta-base') # ->768
        
        # Shared fully connected layers
        # self.fc_shared = nn.Sequential(
        #     nn.Linear(128 + 128 + 128 + 512 + 512, 1024), # one 512 removed as resnet is not used
        #     nn.LeakyReLU(0.01),
        #     nn.Dropout(0.3),
        #     nn.Linear(1024, 512),
        #     nn.ReLU(),
        #     nn.Dropout(0.3),
        # )

        # Shared fully connected layers with improved initialization and normalization
        self.fc_shared = nn.Sequential(
            nn.Linear(128 + 128 + 128 + 512+512, 1024),  # Adjusted input dimensions
            nn.BatchNorm1d(1024),  # Add BatchNorm to stabilize training
            nn.LeakyReLU(negative_slope=0.01),  # LeakyReLU for smoother gradients
            nn.Dropout(0.2),  # Reduced dropout for regularization
            
            nn.Linear(1024, 512),  # Next layer
            nn.BatchNorm1d(512),  # Normalize activations
            nn.GELU(),  # GELU for smooth non-linearity
            nn.Dropout(0.2),  # Reduced dropout for this layer
        )
        
        # Define fully connected layers for each task
        self.fc_sentiment = nn.Linear(512, 3)  # 3 classes for sentiment
        self.fc_emotion = nn.Linear(512, 10)  # 10 classes for emotion
        self.fc_sarcasm = nn.Linear(512, 1)  # Binary classification for sarcasm
        self.fc_bully = nn.Linear(512, 2)  # Binary classification for bully detection
        self.fc_harmful_score = nn.Linear(512, 3)  # 3 classes for harmful score
        self.fc_target = nn.Linear(512, 4)  # 4 classes for target
        
    
    def forward(self, image, image_clip_input, image_vgg_feature, text_clip_input, text_roberta_embedding):
    
        #use BERT + VGG19 combination
        
        img_clip_out = self.image_clip_model(image_clip_input) # 512 ->128
        text_clip_out = self.text_clip_model(text_clip_input) #512 ->128
        text_roberta_out = self.text_roberta_model(text_roberta_embedding) # 768 -> 128
        #img_resnet_out = self.resnet_model(image) # 3x224x224 -> 512
        vgg19_out = self.vgg_dense(image_vgg_feature) #25088 -> 512 # in_CI
        
        self_atten_bilstm_out = self.attention_bilstm_model(text_roberta_embedding) #768 -> 512
        #128 + 128+ 128+ 512 + 512 +512
        
#         print("img_clip_out shape:", img_clip_out.shape)
#         print("text_clip_out shape:", text_clip_out.shape)
#         print("text_roberta_out shape:", text_roberta_out.shape)
#         print("img_resnet_out shape:", img_resnet_out.shape)
#         print("vgg19_out shape:", vgg19_out.shape)
#         print("self_atten_bilstm_out shape:", self_atten_bilstm_out.shape)
        
        # Add a batch dimension to vgg19_out
        # if vgg19_out.dim() == 1:  # Check if it has only one dimension
            # vgg19_out = vgg19_out.unsqueeze(0)  # Shape: (1, 512)
            
        # Squeeze the second dimension to make all tensors 2D
        img_clip_out = img_clip_out.squeeze(1)  # Shape: (8, 128)
        text_clip_out = text_clip_out.squeeze(1)  # Shape: (8, 128)
        text_roberta_out = text_roberta_out.squeeze(1)  # Shape: (8, 128)
        
        combined_features = torch.cat((img_clip_out, text_clip_out, text_roberta_out, vgg19_out, self_atten_bilstm_out), dim = 1)
        
        features = self.fc_shared(combined_features)
        
        # Pass through fully connected layers for each task
        sentiment_output = self.fc_sentiment(features)
        emotion_output = self.fc_emotion(features)
        sarcasm_output = self.fc_sarcasm(features)
        bully_output = self.fc_bully(features)
        harmful_score_output = self.fc_harmful_score(features)
        target_output = self.fc_target(features)
        
        # Task-specific outputs with softmax or sigmoid
#         sentiment_output = F.softmax(self.fc_sentiment(features), dim=1)  # 3-class softmax
#         emotion_output = F.softmax(self.fc_emotion(features), dim=1)      # 10-class softmax
#         sarcasm_output = torch.sigmoid(self.fc_sarcasm(features))         # Binary sigmoid
#         bully_output = F.softmax(self.fc_bully(features), dim=1)          # Binary softmax
#         harmful_score_output = F.softmax(self.fc_harmful_score(features), dim=1)  # 3-class softmax
#         target_output = F.softmax(self.fc_target(features), dim=1)        # 4-class softmax
        
        return sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output
                        
    

In [33]:
# Sanity Check for Model 
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

#Example usage
img_clip_inp_dim = 512
vgg_inp_dim = 25088
text_clip_inp_dim = 512
text_roberta_inp_dim = 768

model = MemeModelCLIPVGG(img_clip_inp_dim, vgg_inp_dim, text_clip_inp_dim, text_roberta_inp_dim) # Create the dataset and dataloader
dataset = MemeDataset(df, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
#model= nn.DataParallel(model)
model.to(device)

# load dtaset and test
data_sample = dataset_clip_vgg[0]
image = data_sample['image'].unsqueeze(0).to(device) # be careful
image_clip_input = data_sample['image_clip_input'].to(device)
image_vgg_feature = data_sample['image_vgg_feature'].to(device)
text_clip_input = data_sample['text_clip_input'].to(device)
text_roberta_embedding = data_sample['text_roberta_embedding'].to(device)

sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output = model(image, image_clip_input, image_vgg_feature, text_clip_input, text_roberta_embedding )

Device: cuda:0


NameError: name 'MemeDataset' is not defined

In [41]:
import torch
torch.cuda.empty_cache()

In [40]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  15363 MiB |  15373 MiB |  34340 MiB |  18977 MiB |\n|       from large pool |  15295 MiB |  15300 MiB |  27058 MiB |  11763 MiB |\n|       from small pool |     68 MiB |     73 MiB |   7282 MiB |   7213 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  15363 MiB |  15373 MiB |  34340 MiB |  18977 MiB |\n|       from large pool |  15295 MiB |  15300 MiB |

# Training

## Load Dtaset and Create the Model

In [ ]:
torch.cuda.device_count()

In [21]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset (assume the dataframe and image directory are available)
dataset_clip_vgg = MemeDatasetClipVGG(df, transform, clip_model, vgg19)

train_size = int(0.8 * len(dataset_clip_vgg))
val_size = len(dataset_clip_vgg) - train_size

# Split Dataset
train_dataset, val_dataset = random_split(dataset_clip_vgg, [train_size, val_size])

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Initialize the model and move it to the GPU

img_clip_inp_dim = 512
vgg_inp_dim = 25088
text_clip_inp_dim = 512
text_roberta_inp_dim = 768

model = MemeModelCLIPVGG(img_clip_inp_dim, vgg_inp_dim, text_clip_inp_dim, text_roberta_inp_dim)

# Parallelize
# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model)  # Use DataParallel if multiple GPUs are available

model.to(device) 

# Initialize the Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# Make the Loss Functions
# Loss functions
loss_fn_sentiment = nn.CrossEntropyLoss().to(device)
loss_fn_emotion = nn.CrossEntropyLoss().to(device)
loss_fn_sarcasm = nn.BCEWithLogitsLoss().to(device)
loss_fn_bully = nn.CrossEntropyLoss().to(device)
loss_fn_harmful_score = nn.CrossEntropyLoss().to(device)
loss_fn_target = nn.CrossEntropyLoss().to(device)


In [23]:
def train_model(model, train_dataloader, val_dataloader, optimizer, losses, vals, num_epochs=10):
    torch.cuda.empty_cache()
    #tqdm(range(num_epochs), position=0, leave=True)
    for epoch in range(num_epochs):
        torch.cuda.empty_cache()
        model.train()
        train_running_loss = 0.0
        batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
        for data_sample in batch_iterator:
            #extract features
            image = data_sample['image'].to(device)
            image_clip_input = data_sample['image_clip_input'].to(device)
            image_vgg_feature = data_sample['image_vgg_feature'].to(device)
            text_clip_input = data_sample['text_clip_input'].to(device)
            text_roberta_embedding = data_sample['text_roberta_embedding'].to(device)

            #extract lables
            sentiment_labels = data_sample['sentiment'].to(device)
            emotion_labels = data_sample['emotion'].to(device)
            sarcasm_labels = data_sample['sarcasm'].to(device)
            bully_labels = data_sample['bully_label'].to(device)
            harmful_score_labels = data_sample['harmful-score'].to(device)
            target_labels = data_sample['target'].to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward Pass
            sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output = model(image, image_clip_input, image_vgg_feature, text_clip_input, text_roberta_embedding )

            # Calculate Loss
            # Compute the losses
            loss_sentiment = loss_fn_sentiment(sentiment_output, sentiment_labels)
            loss_emotion = loss_fn_emotion(emotion_output, emotion_labels)
            loss_sarcasm = loss_fn_sarcasm(sarcasm_output.squeeze(), sarcasm_labels.float())
            loss_bully = loss_fn_bully(bully_output, bully_labels)
            loss_harmful_score = loss_fn_harmful_score(harmful_score_output, harmful_score_labels)
            loss_target = loss_fn_target(target_output, target_labels)

            # Combine the losses
            loss = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Update running loss
            train_running_loss += loss.item()
        
        # Print the average loss for this epoch
        train_running_loss = train_running_loss / len(train_dataloader)
        losses.append(train_running_loss)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_running_loss:.4f}')
    
        
            

In [ ]:
import os

#os.environ["PYTORCH_CUDA_ALLOC_CONF"]= "expandable_segments:True"

In [24]:
losses = []
vals = []
train_model(model, train_dataloader, val_dataloader, optimizer, losses, vals, num_epochs=20)

Processing Epoch 00:   0%|          | 0/308 [00:00<?, ?it/s]/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(image.unsqueeze(0)).to(device)
Processing Epoch 00: 100%|██████████| 308/308 [01:39<00:00,  3.09it/s]


Epoch 1/20, Loss: 5.1323


Processing Epoch 01: 100%|██████████| 308/308 [01:33<00:00,  3.30it/s]


Epoch 2/20, Loss: 4.3281


Processing Epoch 02: 100%|██████████| 308/308 [01:34<00:00,  3.26it/s]


...

Epoch 18/20, Loss: 0.6195


Processing Epoch 18: 100%|██████████| 308/308 [01:34<00:00,  3.26it/s]


Epoch 19/20, Loss: 0.6003


Processing Epoch 19: 100%|██████████| 308/308 [01:33<00:00,  3.29it/s]

Epoch 20/20, Loss: 0.5952


In [30]:
# Import required libraries (if not already done)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

# Validation
model.eval()
val_running_loss = 0.0

# Initialize lists to store true and predicted labels for each task
all_labels_bully = []
all_preds_bully = []

all_labels_sentiment = []
all_preds_sentiment = []

all_labels_emotion = []
all_preds_emotion = []

all_labels_sarcasm = []
all_preds_sarcasm = []

all_labels_harmful_score = []
all_preds_harmful_score = []

all_labels_target = []
all_preds_target = []

epoch = 20

with torch.no_grad():
    torch.cuda.empty_cache()
    batch_iterator = tqdm(val_dataloader, desc=f"Processing Validation {epoch:02d}")
    for data_sample in batch_iterator:
        #extract features
        image = data_sample['image'].to(device)
        image_clip_input = data_sample['image_clip_input'].to(device)
        image_vgg_feature = data_sample['image_vgg_feature'].to(device)
        text_clip_input = data_sample['text_clip_input'].to(device)
        text_roberta_embedding = data_sample['text_roberta_embedding'].to(device)

        #extract lables
        sentiment_labels = data_sample['sentiment'].to(device)
        emotion_labels = data_sample['emotion'].to(device)
        sarcasm_labels = data_sample['sarcasm'].to(device)
        bully_labels = data_sample['bully_label'].to(device)
        harmful_score_labels = data_sample['harmful-score'].to(device)
        target_labels = data_sample['target'].to(device)
        
        # Forward Pass
        sentiment_output, emotion_output, sarcasm_output, bully_output, harmful_score_output, target_output = model(image, image_clip_input, image_vgg_feature, text_clip_input, text_roberta_embedding )

        # Calculate Loss
        # Compute the losses
        loss_sentiment = loss_fn_sentiment(sentiment_output, sentiment_labels)
        loss_emotion = loss_fn_emotion(emotion_output, emotion_labels)
        loss_sarcasm = loss_fn_sarcasm(sarcasm_output.squeeze(), sarcasm_labels.float())
        loss_bully = loss_fn_bully(bully_output, bully_labels)
        loss_harmful_score = loss_fn_harmful_score(harmful_score_output, harmful_score_labels)
        loss_target = loss_fn_target(target_output, target_labels)
        
        # Combine the losses
        loss = loss_sentiment + loss_emotion + loss_sarcasm + loss_bully + loss_harmful_score + loss_target
        
        # Update running loss
        val_running_loss += loss.item()

        # Get predictions for each task
        _, predicted_sentiment = torch.max(sentiment_output, 1)
        _, predicted_emotion = torch.max(emotion_output, 1)
        _, predicted_sarcasm = torch.max(sarcasm_output, 1)
        _, predicted_bully = torch.max(bully_output, 1)
        _, predicted_harmful_score = torch.max(harmful_score_output, 1)  # Assuming multi-class
        _, predicted_target = torch.max(target_output, 1)  # Assuming multi-class

        # Collect true and predicted labels for each task
        all_labels_sentiment.append(sentiment_labels.cpu().numpy())
        all_preds_sentiment.append(predicted_sentiment.cpu().numpy())

        all_labels_emotion.append(emotion_labels.cpu().numpy())
        all_preds_emotion.append(predicted_emotion.cpu().numpy())

        all_labels_sarcasm.append(sarcasm_labels.cpu().numpy())
        all_preds_sarcasm.append(predicted_sarcasm.cpu().numpy())

        all_labels_bully.append(bully_labels.cpu().numpy())
        all_preds_bully.append(predicted_bully.cpu().numpy())

        all_labels_harmful_score.append(harmful_score_labels.cpu().numpy())
        all_preds_harmful_score.append(predicted_harmful_score.cpu().numpy())

        all_labels_target.append(target_labels.cpu().numpy())
        all_preds_target.append(predicted_target.cpu().numpy())
        
    # Print the average loss for this epoch
    val_running_loss = val_running_loss / len(train_dataloader)
    vals.append(val_running_loss)
    print(f'Epoch 20, Val_Loss: {val_running_loss:.4f}')


# Flatten lists for each task
all_labels_bully = np.concatenate(all_labels_bully)
all_preds_bully = np.concatenate(all_preds_bully)

all_labels_sentiment = np.concatenate(all_labels_sentiment)
all_preds_sentiment = np.concatenate(all_preds_sentiment)

all_labels_emotion = np.concatenate(all_labels_emotion)
all_preds_emotion = np.concatenate(all_preds_emotion)

all_labels_sarcasm = np.concatenate(all_labels_sarcasm)
all_preds_sarcasm = np.concatenate(all_preds_sarcasm)

all_labels_harmful_score = np.concatenate(all_labels_harmful_score)
all_preds_harmful_score = np.concatenate(all_preds_harmful_score)

all_labels_target = np.concatenate(all_labels_target)
all_preds_target = np.concatenate(all_preds_target)

# Calculate accuracy and F1 score for each task
accuracy_bully_SA_EM = accuracy_score(all_labels_bully, all_preds_bully)
f1_bully_SA_EM = f1_score(all_labels_bully, all_preds_bully, average='weighted')

accuracy_sentiment_SA_EM = accuracy_score(all_labels_sentiment, all_preds_sentiment)
f1_sentiment_SA_EM = f1_score(all_labels_sentiment, all_preds_sentiment, average='weighted')

accuracy_emotion_SA_EM = accuracy_score(all_labels_emotion, all_preds_emotion)
f1_emotion_SA_EM = f1_score(all_labels_emotion, all_preds_emotion, average='weighted')

accuracy_sarcasm_SA_EM = accuracy_score(all_labels_sarcasm, all_preds_sarcasm)
f1_sarcasm_SA_EM = f1_score(all_labels_sarcasm, all_preds_sarcasm, average='weighted')

accuracy_harmful_score_SA_EM = accuracy_score(all_labels_harmful_score, all_preds_harmful_score)
f1_harmful_score_SA_EM = f1_score(all_labels_harmful_score, all_preds_harmful_score, average='weighted')

accuracy_target_SA_EM = accuracy_score(all_labels_target, all_preds_target)
f1_target_SA_EM = f1_score(all_labels_target, all_preds_target, average='weighted')

print(f'Epoch {epoch}, Validation Loss: {val_running_loss:.4f},\n'
      f'Bully Accuracy: {accuracy_bully_SA_EM:.4f}, F1 Score: {f1_bully_SA_EM:.4f},\n'
      f'Sentiment Accuracy: {accuracy_sentiment_SA_EM:.4f}, F1 Score: {f1_sentiment_SA_EM:.4f},\n'
      f'Emotion Accuracy: {accuracy_emotion_SA_EM:.4f}, F1 Score: {f1_emotion_SA_EM:.4f},\n'
      f'Sarcasm Accuracy: {accuracy_sarcasm_SA_EM:.4f}, F1 Score: {f1_sarcasm_SA_EM:.4f},\n'
      f'Harmful Score Accuracy: {accuracy_harmful_score_SA_EM:.4f}, F1 Score: {f1_harmful_score_SA_EM:.4f},\n'
      f'Target Accuracy: {accuracy_target_SA_EM:.4f}, F1 Score: {f1_target_SA_EM:.4f}')




Processing Validation 20:   0%|          | 0/77 [00:00<?, ?it/s]/tmp/ipykernel_30/2376718173.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(image.unsqueeze(0)).to(device)
Processing Validation 20: 100%|██████████| 77/77 [00:15<00:00,  5.00it/s]


Epoch 20, Val_Loss: 2.1742
Epoch 20, Validation Loss: 2.1742,
Bully Accuracy: 0.7841, F1 Score: 0.7491,
Sentiment Accuracy: 0.6640, F1 Score: 0.6392,
Emotion Accuracy: 0.2532, F1 Score: 0.2359,
Sarcasm Accuracy: 0.5016, F1 Score: 0.3351,
Harmful Score Accuracy: 0.9821, F1 Score: 0.9813,
Target Accuracy: 0.7630, F1 Score: 0.7145
